In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [2]:
PATH = 'data/doto/'

In [3]:
df_raw = pd.read_csv(f'{PATH}train9.csv', low_memory=False)
df_raw=df_raw.drop('num_wins',axis=1)

In [4]:
train_cats(df_raw)

In [5]:
df_raw.head()

,user_id,hero_id,id,num_games,kda_ratio
0,1,74,1_74,304,3743.699732
1,1,65,1_65,255,4009.021513
2,1,39,1_39,203,5094.621514
3,1,106,1_106,158,4619.212963
4,1,2,1_2,157,2871.627907


In [6]:
df, y, nas = proc_df(df_raw, 'kda_ratio')

In [7]:
for col in ['user_id', 'hero_id']:
    df[col] = df[col].astype('category')

In [8]:
m = RandomForestRegressor(n_estimators=350,n_jobs=-1, oob_score=True)
m.fit(df, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [9]:
train_raw = pd.read_csv(f'{PATH}train1.csv', low_memory=False)
train_raw = train_raw.drop('num_wins',axis=1)

In [10]:
train_cats(train_raw)

In [11]:
train_raw.head()

,user_id,hero_id,id,num_games,kda_ratio
0,1,41,1_41,201,3731.707317
1,2,17,2_17,427,2922.058381
2,5,106,5_106,240,5116.682739
3,6,65,6_65,206,4131.553861
4,7,69,7_69,697,2474.469306


In [12]:
df_valid, y_valid, nas = proc_df(df_raw, 'kda_ratio')

In [13]:
for col in ['user_id', 'hero_id']:
    df_valid[col] = df_valid[col].astype('category')

In [14]:
preds = m.predict(df_valid)

In [15]:
preds

array([3751.11359, 3935.36498, 4486.40238, ..., 4116.62197, 4777.92727, 3944.57297])

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_valid, preds))

In [17]:
rms

356.3034191722753

In [18]:
test_df_raw = pd.read_csv(f'{PATH}test9.csv', low_memory=False)
test_df_raw = test_df_raw.drop('num_wins',axis=1)

In [19]:
train_cats(test_df_raw)

In [20]:
test_df, test_y, nas = proc_df(test_df_raw, 'kda_ratio')

In [21]:
for col in ['user_id', 'hero_id']:
    test_df[col] = test_df[col].astype('category')

In [22]:
test_df.head()

,user_id,hero_id,id,num_games
0,3,65,6338,330
1,3,104,6332,231
2,3,7,6340,200
3,3,19,6334,189
4,3,4,6335,182


In [23]:
m.fit(test_df,test_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [24]:
test_raw = pd.read_csv(f'{PATH}test1.csv', low_memory=False)

In [25]:
train_cats(test_raw)

In [26]:
y_temp = np.zeros(len(test_raw))
y_temp.shape

(898,)

In [27]:
test_raw['kda_ratio']=y_temp

In [28]:
test_valid, y_valid, nas = proc_df(test_raw, 'kda_ratio')

In [29]:
for col in ['user_id', 'hero_id']:
    test_valid[col] = test_valid[col].astype('category')

In [30]:
test_valid.head()

,user_id,hero_id,id,num_games
0,3,53,715,278
1,4,17,754,180
2,9,51,898,70
3,10,54,35,189
4,15,74,203,263


In [31]:
preds = m.predict(test_valid)

In [32]:
ids = test_raw['id']

In [34]:
subm = np.stack([ids,preds], axis=1)


In [35]:
submission_file_name = 'doto_kda_pred.csv'
np.savetxt(submission_file_name, subm, fmt='%s,%0.5f', header='id,kda_ratio', comments='')

In [36]:
from IPython.display import FileLink
FileLink(submission_file_name)

/Users/gokulakrishnanm/Desktop/Deeplearning/contests/analytics-vidhya/coursetemp/fastai/courses/ml1/doto_kda_pred.csv